# Genetic Algorithm for Santa's Workshop Optimization

## 1. Algorithm Overview
- **Objective**: Assign 5,000 families to 100 workshop days
  - Minimize: `Preference Costs` + `Accounting Penalties`
  - Constraints: 125 ≤ Daily Occupancy ≤ 300

- **Evolutionary Approach**:
  - Population of candidate solutions
  - Iterative improvement via selection, crossover, and mutation

## 2. Key Components

### Solution Representation
- **Encoding**: List of 5,000 integers (days 1-100)
  - Index = Family ID
  - Value = Assigned day

### Fitness Evaluation
- Reuses cost functions from Artificial Immune system solution:
  ```python
  def calculate_affinity(solution):
      return (calculate_preference_cost(solution) + 
              calculate_accounting_penalty(solution))
- These functions precisely encode Santa's workshop's official requirements
- Using identical functions enables fair comparison between GA and AIS

In [35]:
import csv

with open('family_data.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    headers = next(reader)
    family_data = [list(map(int, line)) for line in reader]

In [36]:
import random
import numpy as np
from typing import List, Tuple
from copy import deepcopy

num_families = 5000
num_days = 100
min_occupancy = 125
max_occupancy = 300


In [37]:
def calculate_preference_cost(solution):
    total_cost = 0
    for family_id, assigned_day in enumerate(solution):
        family = family_data[family_id]
        choices = family[1:-1]
        n_people = family[-1]
        
        # Determining which choice corresponds to the assigned day
        choice_index = choices.index(assigned_day) if assigned_day in choices else len(choices)
        
        # Calculating the cost of gifts
        if choice_index == 0:
            cost = 0
        elif choice_index == 1:
            cost = 50
        elif choice_index == 2:
            cost = 50 + 9 * n_people
        elif choice_index == 3:
            cost = 100 + 9 * n_people
        elif choice_index == 4:
            cost = 200 + 9 * n_people
        elif choice_index == 5:
            cost = 200 + 18 * n_people
        elif choice_index == 6:
            cost = 300 + 18 * n_people
        elif choice_index == 7:
            cost = 300 + 36 * n_people
        elif choice_index == 8:
            cost = 400 + 36 * n_people
        elif choice_index == 9:
            cost = 500 + 235 * n_people 
        else:
            cost = 500 + 434 * n_people
            
        total_cost += cost
        
    return total_cost

def calculate_accounting_penalty(solution):
    daily_occupancy = [0] * num_days
    for family_id, assigned_day in enumerate(solution):
        n_people = family_data[family_id][-1]
        daily_occupancy[assigned_day - 1] += n_people
    
    penalty = 0
    for d in range(num_days):
        Nd = daily_occupancy[d]
        Nd_next = daily_occupancy[d + 1] if d < num_days - 1 else Nd
        penalty += ((Nd - 125) / 400) * Nd**(0.5 + abs(Nd - Nd_next) / 50)
    return penalty

def calculate_affinity(solution):
    return calculate_preference_cost(solution) + calculate_accounting_penalty(solution)

# Population Initialization Functions

## `initialize_individual() -> List[int]`
Creates a single valid solution using a constraint-aware greedy approach.

### Algorithm Steps:
1. **Sort families** by size (descending) to handle largest groups first
2. **Initialize daily occupancy tracker** (`[0] * 100`)
3. For each family:
   - Extract their preferred days (`choices = family[1:-1]`)
   - Get family size (`n_people = family[-1]`)
   - Find valid preferred days (those that won't exceed `max_occupancy`)
   - Assign to:
     - Random valid preferred day (if available)
     - Random valid fallback day (if no preferred days work)
     - Completely random day (last resort)
   - Update occupancy counts

### Key Features:
- **Constraint Satisfaction**: Guarantees daily occupancy ≤ 300
- **Preference Awareness**: Prioritizes families' preferred days when possible
- **Deterministic Start**: Sorting ensures reproducible initialization

## `initialize_population(pop_size: int) -> List[List[int]]`
Generates multiple diverse starting solutions.

In [38]:
def initialize_individual() -> List[int]:
    """Create one valid individual using greedy approach"""
    solution = []
    daily_occupancy = [0] * num_days
    
    # Sort families by size (largest first)
    sorted_families = sorted(family_data, key=lambda x: x[-1], reverse=True)
    
    for family in sorted_families:
        choices = family[1:-1]
        n_people = family[-1]
        
        # Try preferred days first
        valid_choices = [day for day in choices 
                        if daily_occupancy[day - 1] + n_people <= max_occupancy]
        
        if valid_choices:
            assigned_day = random.choice(valid_choices)
        else:
            # Fallback to any valid day
            valid_days = [day for day in range(1, num_days + 1) 
                         if daily_occupancy[day - 1] + n_people <= max_occupancy]
            assigned_day = random.choice(valid_days) if valid_days else random.randint(1, num_days)
        
        solution.append(assigned_day)
        daily_occupancy[assigned_day - 1] += n_people
    
    return solution

def initialize_population(pop_size: int) -> List[List[int]]:
    """Create initial population"""
    return [initialize_individual() for _ in range(pop_size)]

# Selection Functions

## `tournament_selection()`
- **Purpose**: Selects one solution via competitive tournament
- **Mechanism**:
  1. Randomly picks `tournament_size` solutions
  2. Returns the solution with lowest fitness (best score)
- **Parameters**:
  - `tournament_size=3`: Optimal balance between selection pressure and diversity
- **Output**: Single best solution from tournament

## `select_parents()`
- **Purpose**: Combines elitism and tournament selection
- **Process**:
  1. **Elitism Phase**:
     - Selects top `elite_size` solutions by fitness
     - Preserves them unchanged
  2. **Tournament Phase**:
     - Fills remaining spots via tournament selection
- **Parameters**:
  - `elite_size`: Number of top solutions to protect (typically 2-5)
- **Output**: Tuple of (elite solutions, tournament-selected parents)

In [39]:
def tournament_selection(population: List[List[int]], fitnesses: List[float], 
                        tournament_size: int = 3) -> List[int]:
    """Select one individual using tournament selection"""
    contestants = random.sample(list(zip(population, fitnesses)), tournament_size)
    return min(contestants, key=lambda x: x[1])[0]

def select_parents(population: List[List[int]], fitnesses: List[float], 
                   elite_size: int = 0) -> Tuple[List[List[int]], List[List[int]]]:
    """Select parents using elitism + tournament selection"""
    # Elite selection
    elite = []
    if elite_size > 0:
        elite_indices = np.argsort(fitnesses)[:elite_size]
        elite = [population[i] for i in elite_indices]
    
    # Tournament selection for remaining parents
    parents = []
    for _ in range(len(population) - elite_size):
        parents.append(tournament_selection(population, fitnesses))
    
    return elite, parents

# Crossover Operators

## `uniform_crossover()`
- **Type**: Gene-wise mixing
- **Operation**:
  - For each family, randomly selects assignment from either parent
  - Independent 50/50 chance for each gene

## `day_block_crossover()`
- **Type**: Segment swapping
- **Operation**:
  - Divides solution into blocks of `block_size` families
  - For each block: 50% chance to swap parent segments
- **Parameters**:
  - `block_size=100`: Balances exploration/exploitation

In [40]:
def uniform_crossover(parent1: List[int], parent2: List[int]) -> List[int]:
    """Uniform crossover - each gene comes from either parent"""
    return [random.choice([p1, p2]) for p1, p2 in zip(parent1, parent2)]

def day_block_crossover(parent1: List[int], parent2: List[int], 
                       block_size: int = 100) -> List[int]:
    """Crossover blocks of days together"""
    child = parent1.copy()
    # Swap random blocks of family assignments
    for i in range(0, num_families, block_size):
        if random.random() < 0.5:  # 50% chance to swap block
            start, end = i, min(i + block_size, num_families)
            child[start:end] = parent2[start:end]
    return child

# Local Search Function

## `local_search(solution)`
**Purpose**: Repairs invalid solutions by adjusting family assignments to meet occupancy constraints.

### Key Operations:
1. **Occupancy Calculation**:
   - Tracks daily visitor counts
   - Identifies under/over-occupied days

2. **Constraint Repair**:
   - **For under-occupied days (<125)**:
     - Finds movable families from over-occupied days
     - Transfers them while maintaining other constraints
   - **For over-occupied days (>300)**:
     - Randomly reassigns families to valid days
     - Ensures new assignments don't create other violations

### Behavior Flow:
1. Calculate current daily occupancies
2. For each day:
   - While constraints violated:
     - Find and execute valid family moves
     - Break if no improvements possible
3. Return repaired solution

In [41]:
def local_search(solution):
    daily_occupancy = [0] * num_days
    for family_id, day in enumerate(solution):
        daily_occupancy[day - 1] += family_data[family_id][-1]
    
    for day in range(100):
        while daily_occupancy[day] < min_occupancy or daily_occupancy[day] > max_occupancy:
            if daily_occupancy[day] < min_occupancy:
                # Finding families that can be moved on this day
                for family_id, assigned_day in enumerate(solution):
                    family = family_data[family_id]
                    n_people = family[-1]
                    
                    # We check whether it is possible to move the family on this day
                    if (daily_occupancy[day] + n_people <= max_occupancy and
                        daily_occupancy[assigned_day - 1] - n_people >= min_occupancy):
                        # Moving the family
                        solution[family_id] = day + 1
                        daily_occupancy[assigned_day - 1] -= n_people
                        daily_occupancy[day] += n_people
                        break
                else:
                    break
            
            elif daily_occupancy[day] > max_occupancy:
                for family_id, assigned_day in enumerate(solution):
                    if assigned_day == day + 1:
                        family = family_data[family_id]
                        n_people = family[-1]
                        
                        valid_days = [d for d in range(1, num_days + 1) if daily_occupancy[d - 1] + n_people <= max_occupancy]
                        if valid_days:
                            new_day = random.choice(valid_days)
                            solution[family_id] = new_day
                            daily_occupancy[day] -= n_people
                            daily_occupancy[new_day - 1] += n_people
                            break
                else:
                    break
    
    return solution

# Solution Validation and Mutation Functions

## `is_valid(solution)`
**Purpose**: Verifies if a solution meets all problem constraints.

### Key Features:
- **Constraint Checking**:
  - Calculates daily occupancy counts
  - Validates all days have 125-300 visitors
- **Return Value**:
  - `True`: Solution meets all occupancy constraints
  - `False`: At least one day violates constraints

## `mutate(individual, mutation_rate=0.05)`
**Purpose**: Creates genetic diversity by randomly modifying solutions.

### Mutation Process:
1. **Initialization**:
   - Creates copy of input solution
2. **Gene Modification**:
   - For each family (5% chance by default):
     - Preferentially mutates to other preferred days
     - Avoids invalid moves that would exceed max occupancy
3. **Repair**:
   - Automatically applies `local_search()` if mutation creates constraint violations

### Parameters:
- `mutation_rate=0.05`: 
  - Typically mutates ~250 families (for 5000 families)
  - Balances exploration vs solution preservation

In [42]:
def is_valid(solution):
    daily_occupancy = [0] * num_days
    for family_id, day in enumerate(solution):
        daily_occupancy[day - 1] += family_data[family_id][-1]

    return all(min_occupancy <= occupancy <= max_occupancy for occupancy in daily_occupancy)

In [43]:
def mutate(individual: List[int], mutation_rate: float = 0.05) -> List[int]:
    """Mutate an individual by changing some family assignments"""
    mutated = individual.copy()
    for family_id in range(num_families):
        if random.random() < mutation_rate:
            family = family_data[family_id]
            choices = family[1:-1]
            n_people = family[-1]
            
            # Try preferred days first
            current_day = mutated[family_id]
            valid_choices = [day for day in choices 
                            if day != current_day]
            
            if valid_choices:
                mutated[family_id] = random.choice(valid_choices)
    
    # Use your local search to repair if needed
    if not is_valid(mutated):
        mutated = local_search(mutated)
    return mutated

# Genetic Algorithm Main Function

## `genetic_algorithm()`
**Purpose**: Evolves solutions through selection, crossover, and mutation.

### Parameters:
| Parameter | Default | Description |
|-----------|---------|-------------|
| `pop_size` | 100 | Number of solutions per generation |
| `generations` | 100 | Maximum iterations |
| `crossover_rate` | 0.9 | Probability of combining parents |
| `mutation_rate` | 0.05 | Probability of modifying a family's day |
| `elite_size` | 2 | Number of top solutions preserved unchanged |
| `crossover_type` | 'uniform' | Crossover method ('uniform' or 'day_block') |

### Algorithm Flow:
1. **Initialization**:
   - Creates starting population of valid solutions
   - Tracks best solution and fitness history

2. **Generational Loop**:
   - **Evaluation**: Calculates fitness for all solutions
   - **Elitism**: Preserves top performers
   - **Reproduction**:
     - Selects parents via tournament selection
     - Produces offspring through crossover (90% chance)
     - Applies mutations (5% chance per family)
   - **Population Update**: Combines elites and offspring

3. **Termination**:
   - Returns best solution found and fitness progression
   - Automatic constraint repair maintains validity

### Key Features:
- **Adaptive Evolution**: Balances exploration (mutation) and exploitation (selection)
- **Constraint Preservation**: All operations maintain valid solutions
- **Progress Tracking**: Records best fitness per generation

In [44]:
def genetic_algorithm(pop_size: int = 100, generations: int = 100,
                     crossover_rate: float = 0.9, mutation_rate: float = 0.05,
                     elite_size: int = 2, crossover_type: str = 'uniform') -> Tuple[List[int], List[float]]:
    """Run the genetic algorithm"""
    # Initialize population
    population = initialize_population(pop_size)
    best_individual = None
    best_fitness = float('inf')
    fitness_history = []
    
    for gen in range(generations):
        # Evaluation
        fitnesses = [calculate_affinity(ind) for ind in population]
        current_best = min(fitnesses)
        fitness_history.append(current_best)
        
        # Update best solution
        if current_best < best_fitness:
            best_index = np.argmin(fitnesses)
            best_individual = deepcopy(population[best_index])
            best_fitness = current_best
        
        # Selection
        elite, parents = select_parents(population, fitnesses, elite_size)
        
        # Reproduction
        offspring = []
        for i in range(0, len(parents), 2):
            parent1, parent2 = parents[i], parents[i+1] if i+1 < len(parents) else parents[0]
            
            # Crossover
            if random.random() < crossover_rate:
                if crossover_type == 'uniform':
                    child1 = uniform_crossover(parent1, parent2)
                    child2 = uniform_crossover(parent2, parent1)
                else:
                    child1 = day_block_crossover(parent1, parent2)
                    child2 = day_block_crossover(parent2, parent1)
            else:
                child1, child2 = parent1.copy(), parent2.copy()
            
            # Mutation
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)
            
            offspring.extend([child1, child2])
        
        # Form new population
        population = elite + offspring[:pop_size - len(elite)]
        
        # Progress reporting
        print(f"Generation {gen}: Best = {current_best:.2f}, Avg = {np.mean(fitnesses):.2f}")
    
    return best_individual, fitness_history

In [45]:
# Run the GA
best_solution_ga, ga_history = genetic_algorithm(
    pop_size=100,
    generations=100,
    crossover_rate=0.85,
    mutation_rate=0.03,
    elite_size=2,
    crossover_type='uniform'
)

# Compare with your AIS results
print(f"GA Best Solution Cost: {calculate_affinity(best_solution_ga)}")

Generation 0: Best = 7159530528.64, Avg = 15805711426004620.00
Generation 1: Best = 188409629.92, Avg = 4998273243.03
Generation 2: Best = 36395158.33, Avg = 2783346448.34
Generation 3: Best = 34301806.84, Avg = 1941760303.92
Generation 4: Best = 19895969.42, Avg = 1078127794.94
Generation 5: Best = 12706174.54, Avg = 692863449.47
Generation 6: Best = 12706174.54, Avg = 433503575.28
Generation 7: Best = 11355987.99, Avg = 350864509.23
Generation 8: Best = 11355987.99, Avg = 247665957.88
Generation 9: Best = 11355987.99, Avg = 213626265.46
Generation 10: Best = 11355987.99, Avg = 244680666.80
Generation 11: Best = 11355987.99, Avg = 98099379.13
Generation 12: Best = 10206558.68, Avg = 68475385.65
Generation 13: Best = 10206558.68, Avg = 75992302.55
Generation 14: Best = 10127635.43, Avg = 55124654.81
Generation 15: Best = 9460173.30, Avg = 64184757.75
Generation 16: Best = 9460173.30, Avg = 100920529.84
Generation 17: Best = 8481186.15, Avg = 50681486.63
Generation 18: Best = 8481186.15